In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scp
import nltk
import imageio
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn import metrics 
from sklearn.metrics import pairwise_distances 

from timeit import default_timer as timer
%matplotlib inline

import json as js

- Reverse PCA
- polish this stuff
- make python scrips
- make readme nice

# Project
### Stages

- 4x20x10000 stuctures with 19-30 atoms in a cluster
    - coordinates
    - dipole moment
    - energy
- Goals
    - End user
        - Configuration Sampling 
            - Find one structure with the lowest energy
    - Eliminate redundant structures
    - Find descriptors
        - reduce dimensions
    

# The Data

To use this you should provide:  
- the **path** to your XTB-data
- the **name** for your data

> The name will be used to name the folder where data produced by this code is put

In [ ]:
XTB_path = input("Where your XTB-data is?")
theName = input("Your data will be in:")

In [ ]:
print(type(XTB_path),XTB_path)
print(type(theName),theName)


In [ ]:
!echo $XTB_path
!echo $theName

In the notebook user specifies the path of where the XTB data is and what is the name of the data (for the folder)

In python scripts the user is promted about where the XTB data is 

# How to Cluster

#### Read data in

Data must have a `.csv`-file that contains:  
Header row: ``` Filename,LogPath,XYZPath,Dipole,Energy ```


| Filename | LogPath | XYZPath | Dipole| Energy |
|------|------|------|------|------|
| Name of the cluster | path to .log-file | path to .xyz-file | Dipole moment value | Energy value |

> Here we either require the .log-file or remove it from the code

#### Make the csv-file 
> Instructions how to make the .csv-file from the actual data using the script

#### Run these cells with ```shift+enter```

In [ ]:
def init_DF(csvfile):
    "Reads a xyz-file and initializes the dataframe."
    
    filescsv = pd.read_csv(csvfile)                    # Read tthe .csv-file to a DataFrame
    xyz_temp = filescsv.iloc[0,2]                      # Get one .xyz-file as a reference
    
    xyz_test = read_xyz(xyz_temp,flatten=False)        # Read coordinates to a DF (no flattening)
    
    #filenames = filescsv.iloc[:,0]                    # Get the names for files as a Series (This one not used here)
    n_atoms = xyz_test.index.size                      # Number of atoms from the reference .xyz-file
    atomnames = np.array(xyz_test.index)               # Names of all the atoms as ndarray
    
    list_atomnames = []
    for i,j in enumerate(atomnames):                   # Give numbers for the atoms
        list_atomnames.append("{}_{}".format(j,i))
    arr_atomnames = np.array(list_atomnames)           # Enumerated atomnames as an ndarray
    
    col_names = makeMultiIndexs(arr_atomnames)         # Make column names (multi-index)

    arr_clusters,filenames = init_xyz_array(filescsv,n_atoms) # Read the actual coordinate data
    
    coord_df = pd.DataFrame(arr_clusters,index=filenames,columns=col_names) # Save the data in nice DF-fornat
    
    
    return coord_df

In [ ]:
def init_xyz_array(filecsv, n_atoms):
    """Reads .xyz-file paths from .csv-file and returns 
    an array with all xyz-coords in rows, and the filenames for DF indexes"""
    
    arr_clusters = np.zeros((1,3*n_atoms+2))             # cols of coord array should be of shape [clusters,atoms*[x,y,z]]
    filenames = np.array(filecsv.Filename)               # Get the names of the clusters from the .csv-file

    for index, row in filecsv.iterrows():                # Iterate through clusters and save their attributes
        dipole = row.Dipole                              # find the dipole value
        energy = row.Energy                              # find the energy value
        metarr = np.array([energy,dipole])               # put dipole and energy to an array
        path = row.XYZPath                               # find the path for .xyz-file
        xyzs = read_xyz(path)                            # Read coordinates to an ndarray with flatten=True (default)

        #if index == None:                               # print stuff for debugging purposes of this function
        #    print("Filename:",filename)
        #    print("metarr:",metarr)
        #    print("shape:",metarr.shape)
        #    print("type:",type(metarr))
        #    print("path:",path)
        #    print("coordshape:",xyzs.shape)
        #    print("coords:",xyzs)
        
        arrr = np.array(list(metarr)+list(xyzs))         # concatenate one cluster info
        arr_clusters = np.vstack((arr_clusters,arrr))    # concatenate clusters to one array
    return arr_clusters[1:,:], filenames

In [ ]:
def read_xyz(xyzfile,flatten=True):
    """
    usage: read_xyz(xyzfile,flatten(Default=True))
    
    Reads a xyz-file and returns the coordinates as DF
    or flattened to a Numpy-array
    """
    xyz_temp = pd.read_csv(xyzfile, \
                       sep='\s+', \
                       skiprows=2, \
                       header=None, \
                       index_col=0, \
                       names=['x','y','z']\
                      ).sort_index(ascending=False)       # Reads coordinates from an xyz-file
    
    atoms3 = xyz_temp.shape[0]*3                          # multiply n of atoms with 3 dimensions
    
    if flatten==True:
        xyz_flat = xyz_temp.values.flatten().reshape(1,atoms3)#.astype(np.float)
        return xyz_flat[0]                                # returns coordinates as ndarray of shape [1,atoms*3]
    else:
        return xyz_temp                                   # returns coordinates as a DF

In [ ]:
def makeMultiIndexs(arr):
    """
    Gets names of atoms as array and makes a pandas multi-indexing structure
    with x-, y- and z- columns for each atom.
    """
    top_arr = np.empty((arr.size * 3 +2)).astype(str)
    top_arr[0] = "Properties"
    top_arr[1] = "Properties"
    top_arr[2::3] = arr
    top_arr[3::3] = arr
    top_arr[4::3] = arr
    butt_arr = np.empty((arr.size * 3 +2)).astype(str)
    butt_arr[0] = "Energy"
    butt_arr[1] = "Dipole"
    butt_arr[2::3] = "x"
    butt_arr[3::3] = "y"
    butt_arr[4::3] = "z"
    col_arr = np.vstack((top_arr,butt_arr))
    #col_arr.transpose()
    col_arr_tuples = list(zip(*col_arr))
    col_names = pd.MultiIndex.from_tuples(col_arr_tuples, names=["Atom","Coord"])
    return col_names

In [ ]:
clusters_df = init_DF("Data2/Data_Collection.csv")

In [ ]:
#from multi-index to one index
clusters_df.columns = ['_'.join(col) for col in clusters_df.columns.values]

## Data exploration / EDA

We start by looking at the correlation among our variables. The correlation betwwen two variables is a measure of the linear dependecy among them:
$$ Cor(X,Y)=\frac{Cov(X,Y)}{\sqrt{Var(X)Var(Y)}}$$
In order to apply a linear model to predict the energy wrt the other variables, is important to check that there is a small correlation  between the variables we are using as predictors, since we will predict the response by an approprate linear combination of the predictors. In our data, there is no particoularly high correlation.

In [ ]:
sns.set(style="white")

# Compute the correlation matrix
corr = clusters_df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11,9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

The correlations among data are not really high, hence we can consider all of hem as independent variables.   
Having a look on the distributions of the energy variable, it is asymmetric, with a tail on the right. 

In [ ]:
sns.set(style="ticks", color_codes=True)
sns.pairplot(clusters_df,  kind="reg",vars=['Properties_Energy','Properties_Dipole'])

Energy and dipole don't have significant correlation

In [ ]:
clusters_df['Properties_Energy'].hist()
plt.title("Energy")
plt.axvline(np.mean(clusters_df['Properties_Energy']), color='k', linestyle='dashed', linewidth=2)

Difference between min and max energies is very small

In [ ]:
sns.catplot( y="Properties_Energy", kind="box", data=clusters_df)

The algorithm in ABCluster tries to make the energies as small as possible hich can be seen here

In [ ]:
energy=clusters_df['Properties_Energy'] # %1 should take only the decimals but does it?
coord=clusters_df.drop(['Properties_Energy','Properties_Dipole'],1) # dipoles and energy out

## Linear regression

In statistics we define as "supervised learning" the set of models that are ment to evaluate an output $y$ (answer variable) as a function of a set of known indipendent variables taken as input $(X_1,...,X_k)$ .   
Furthermore, if we assume that $Y$ has a known probability law, we use parametric models. When $Y$ is a continuous variable, the mainly used model is the regression, that is a method that allows to estimate the conditional average of the answer $ E (Y | X_1 = x_1, ..., X_k = x_k) $ or its transformed as a linear combination of the explanatory $\eta=X '\beta $.
We have a simple linear regression, when using a model of the type:
$$ Y_i = x_i '\beta + \varepsilon_i $$ with $i = 1, ..., n $ number of observations, such that $ n> k $. Equivalently, in compact form:
$$ Y = X \beta + \varepsilon $$ where $ Y_ {nx1} = (Y_1, ..., Y_n) '$ is the vector of n v.c. of independent outputs, $ X_ {nx (k + 1)} $ is the matrix of the drawing, which together with the vector of coefficients $ \beta = (\beta_0, ..., \beta_k) '$ constitutes the deterministic component of the model, while $ \varepsilon = (\varepsilon_1, ..., \varepsilon_n) '$ is the erratic component, where $ \varepsilon_i \sim N (0, \sigma ^ 2 _ {\varepsilon}) $ for each $ i $ are unknown errors with zero mean and constant variance.

We can implement a linear regression model to predict the energy ($y$) of our cluster with rrespect to the coordinates variables ($X$). As we saw above, our response takes all values around -40.9, what makes the difference is the centesimal portion, hence we will focus our attention on predicting this part.


In [ ]:
#divide the dataset in train and test

x_train,x_test,y_train,y_test = train_test_split(coord,energy,test_size=0.20)

In [ ]:
#linear regression model on train set

model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
#model.coeff
r_sq = model.score(x_train, y_train)
r_sq #pretty bad

In [ ]:
#predict from the test
y_pred = model.predict(x_test)
pred_test = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()})
pred_test.head(6)

In [ ]:
#divide the dataset in train and test

coord_train,coord_test,energy_train,energy_test = train_test_split(coord,energy,test_size=0.20)

In [ ]:
#linear regression model on train set

model = LinearRegression()
model.fit(coord_train, energy_train)

In [ ]:
#model.coeff
r_sq = model.score(coord_train, energy_train)
r_sq #pretty bad

Linear regression works poorly which means that the conformation in general doesn't correlate with the energy

In [ ]:
#predict from the test
energy_pred = model.predict(coord_test)
predictions = pd.DataFrame({'Actual': energy_test.values.flatten(), 'Predicted': energy_pred.flatten()})
predictions.head(6)

### Linear Regression with Feature selection

In [ ]:
#feature selection alternative
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

estimator = SVR(kernel="linear")
selector = RFE(estimator, 5, step=1)
selector = selector.fit(coord_train, energy_train)
#print(selector.support_) 
print(selector.ranking_)

> But what is this then selecting?

In [ ]:
coord_train.columns

With only 2 hydrogens the $R^2$ value is smaller than with all the atoms

In [ ]:
#oxigen O8 seems the most significant variable in our model
model1 = LinearRegression().fit(x_train.loc[:,('H_17_x','H_17_y','H_17_z','H_18_x','H_18_y','H_18_z')], y_train) #select the most significant variables...
print('R2 simple reg',model1.score(x_train.loc[:,('H_17_x','H_17_y','H_17_z','H_18_x','H_18_y','H_18_z')], y_train))

With all the other atoms than hydrogens:

In [ ]:
coord_train.iloc[:5,:33]

In [ ]:
# omit hydrogens
model2 = LinearRegression().fit(coord_train.iloc[:,:33], energy_train) #select the most significant variables...
print('R2 simple reg',model2.score(coord_train.iloc[:,:33], energy_train))

## Principal Component Analysis
The PCA is a tool used to reduce the dimensionality of the data.
It tries to preserve as much information as possible.  
These two goals in PCA are pursued by means of a transformation of the original variables into new variables, called Principal Components (PCs).  
They consists of a linear combination of the original variables.
PCs are uncorrelated and arranged in order of decreasing variance, so that the first PCs account for most of the variation in the sample.   

Assuming we want to reduce the number of our original p variables to $k<p$ variables, the PCA method can be formalized as follows:  
• 1st PC: determine the coeffcients of the linear combination
$$Z_{1j} = a_1^T X = \sum_{i=0}^p a_{1i} X_i $$ 
that maximize $ Var(Z_1) = a_1^T \Sigma a_1 $ under the constraint $ a_1^T a_1 = 1 $   

> The formula $ Var(Z_1) = a_1^T \Sigma a_1 $ looks weird

• 2nd PC: determine coeffcients of the linear combination
$$ Z_{2j} = a_2^T X = \sum_{i=0}^p a_{2i} X_i $$
that maximize $ Var(Z_2) = a_2^T \Sigma a_2$ under the constraint $ a_2^T a_2=1$ and $Cov(Z_1,Z_2)=0$   

• proceed in a similar fashion for all other components.   

• Final output: $Z_1, \dots ,Z_p $such that $Var (Z1) > Var (Z2) > \dots > Var (Zp)$ and $cov(Zj ; Zk ) = 0$ for $j \neq k$  

We will apply PCA to our coordinates variable, in order to select new variables, that explain at least 80% of the variability of our data.

In [ ]:
pca = PCA(n_components=0.8,svd_solver='full')   # from docstring: this should include PCs so that 80% of variance is explained
PCs_coord = pca.fit_transform(coord)            # fit_transform ehkä normalisoi
PCs_df = pd.DataFrame(data = PCs_coord, columns=["PC{}".format(i) for i in range(PCs_coord.shape[1])])
print('Explained variance :', np.sum(pca.explained_variance_ratio_)*100, '%')    # It seems that it is 80% :D

In [ ]:
#2D graph of first two PCs
print('Explained variance PC1 :', pca.explained_variance_ratio_[0]*100, '%')
print('Explained variance PC2 :', pca.explained_variance_ratio_[1]*100, '%')

sns.scatterplot(x=PCs_df['PC1'], y=PCs_df['PC2'], hue=energy,  alpha=0.3)
plt.title("2 first PCs of PCA")
plt.ylabel('PC2')
plt.xlabel('PC1')
plt.show()

> Here $\uparrow$ could be some clustering seen but the representation is poor (cartesian coordinates)

## K-means / Clustering
In order to reduce the variety of our observations we are clustering our data, by using a k-mean algorith.  
This algorithm is a non-hierarchical methods of clustering, i.e. we assume the number $ k $ of groups is assumed to be fixed.
Algorithm introduced by MacQueen (1967): each statistical unit is assigned to the cluster whose centroid (i.e. vector of means) is the closest one. The metric used to measure the distance among groups is typically the Euclidean one.

1. Initial partition into K clusters, (possibly randomly generated)
2. For each of the K clusters, compute the cluster centroid.
3. Assign each observation to the cluster whose centroid is closest
4. Recompute centroids for all clusters
5. Repeat 3. - 4. until reaching a maximum number of iterations or when it is not possible to redistribute observations

In order to chose the optimal number of groups, a good criteria is to find a balance between low within-cluster variation $WSS$ and number of groups.  
We start by clustering our original dataset, a clustering of the PCs dtaset is also shown.

#### Energy and dipole

In [ ]:
kmeans_ED = KMeans(n_clusters=4)  #we should choose the number of clusters
kmeans_ED.fit(clusters_df) 
ED_kmeans_res = kmeans_ED.predict(clusters_df)

In [ ]:
plt.scatter(clusters_df['Properties_Energy'], clusters_df['Properties_Dipole'], c=ED_kmeans_res,s=50, cmap='viridis', alpha=0.5)
centers_ED = kmeans_ED.cluster_centers_
plt.scatter(centers_ED[:, 0], centers_ED[:, 1], c='red', s=100, alpha=0.5)
plt.xlabel('Energy')
plt.ylabel('Dipole')
plt.show()

In [ ]:
PCsE_df = pd.concat([PCs_df, clusters_df[['Properties_Energy']]], axis = 1)  
#finalDf = pd.concat([finalDf, clusters_df[['Properties_Dipole']]], axis = 1) # leave dipoles out

#### Coordinates as PCs

In [ ]:
#clustering using PCS as variables
kmeans_PCs = KMeans(n_clusters=4)
kmeans_PCs.fit(PCs_df)
PCs_kmeans = kmeans_PCs.predict(PCs_df)

In [ ]:
plt.scatter(PCs_df['PC1'], PCs_df['PC2'], c=PCs_kmeans,s=3, cmap='viridis', alpha=0.2)
centers_PCs = kmeans_PCs.cluster_centers_
plt.scatter(centers_PCs[:, 0], centers_PCs[:, 1], c='black', s=200, alpha=0.5)
plt.ylabel('PC2')
plt.xlabel('PC1')
plt.show()

#### Coordinates as PCs and Energy

In [ ]:
#clustering using PCS+E as variables
kmeans_PCE = KMeans(n_clusters=23)
kmeans_PCE.fit(PCsE_df)
PCE_kmeans = kmeans_PCE.predict(PCsE_df)

In [ ]:
plt.scatter(PCsE_df['PC1'], PCsE_df['PC2'], c=PCE_kmeans,s=3, cmap='viridis', alpha=0.2)
centers_PCE = kmeans_PCE.cluster_centers_
plt.scatter(centers_PCE[:, 0], centers_PCE[:, 1], c='black', s=200, alpha=0.5)
plt.ylabel('PC2')
plt.xlabel('PC1')
plt.show()

#### Search for optimal k = amount of clusters

In [ ]:
k_values = np.arange(3,40)
ch_values = np.empty(k_values.shape)


In [ ]:
for k in k_values:
    kmeans_model = KMeans(n_clusters=k,random_state=1)
    kmeans_model.fit(PCsE_df)
    labels = kmeans_model.labels_
    ch_values[k-k_values[0]] = metrics.calinski_harabasz_score(PCsE_df,labels)

In [ ]:
k_best = k_values[np.where(ch_values == ch_values.max())[0][0]]

In [ ]:
#clustering using PCS+E as variables with best k-value
kbest_PCE = KMeans(n_clusters=k_best)
kbest_PCE.fit(PCsE_df)
PCE_kbest = kbest_PCE.predict(PCsE_df)

In [ ]:
plt.scatter(PCsE_df['PC1'], PCsE_df['PC2'], c=PCE_kbest,s=3, cmap='viridis', alpha=0.2)
centers_kbest = kbest_PCE.cluster_centers_
plt.scatter(centers_kbest[:, 0], centers_kbest[:, 1], c='black', s=200, alpha=0.5)
plt.ylabel('PC2')
plt.xlabel('PC1')
plt.show()

In [ ]:
labels_PCE = kmeans_PCE.labels_
metrics.calinski_harabasz_score(PCsE_df,labels_PCE)

#### t-SNE for visualization

In [ ]:
tsne_PCE = TSNE(n_components=2).fit_transform(PCsE_df)    # make t-SNE magic happen

In [ ]:
#centers_tSNE = TSNE(n_components=2).fit_transform(centers_PCE)  # Same for centers which doesn't work

```python
tsne = TSNE(n_components=2)
tsne_PCE = tsne.fit_transform(PCsE_df)
centers_tSNE = tsne.fit_transform(centers_PCE)
```
> Does this the same as upper?

In [ ]:
tsne_PCE_cluster = pd.concat((pd.DataFrame(tsne_PCE,columns=['tsne1','tsne2']), pd.DataFrame(PCE_kmeans,columns=['groupID'])), axis = 1)  
# combine t-SNE info with the info of the cluster

In [ ]:
centers_tSNE = tsne_PCE_cluster.groupby(by='groupID').mean()
# calculate the centers as means

In [ ]:
!say "The code is ready"

In [ ]:
plt.scatter(tsne_PCE[:,0],tsne_PCE[:,1], c=PCE_kmeans,s=3, cmap='viridis', alpha=0.3)

plt.scatter(centers_tSNE.iloc[:, 0], centers_tSNE.iloc[:, 1], c='black', s=200, alpha=0.5)
plt.title("Clusters' Cluster")
plt.ylabel('t-SNE component 2')
plt.xlabel('t-SNE component 1')
plt.savefig("broccoli_centers.png",dpi=300)
plt.show()

**Try to get PCA reversed**

In [ ]:
import sklearn.datasets

X = sklearn.datasets.load_iris().data 
print(X[:5])

mu = np.mean(X, axis=0) 
print(mu)

peeceeaa = PCA()
peeceeaa.fit(X)

nComp = 2 
Xhat = np.dot(peeceeaa.transform(X)[:,:nComp], peeceeaa.components_[:nComp,:]) 
print(Xhat[:5])

Xhat += mu 
print(np.around(Xhat[:5],1))

> The stuff forslecting the structures

```python
import numpy as np 
import sklearn.datasets, sklearn.decomposition 
 
X = sklearn.datasets.load_iris().data 
mu = np.mean(X, axis=0) 
 
pca = sklearn.decomposition.PCA() 
pca.fit(X) 
 
nComp = 2 
Xhat = np.dot(pca.transform(X)[:,:nComp], pca.components_[:nComp,:]) 
Xhat += mu 
 
print(Xhat[0,])

to determin the optimal k inmkmeans algo, we can choose the indicator that mazimize this index (Kalinkski and harabatz)


```


#### Look for the nice amount of clusters
from the chem point of view there should be 23 different clusters (permutations of molecules) so that would be nice to try that

***

***

#clustering using PCS and energies as variables and t-SNE for visualization

kmeans_PCs.fit(DF_embedded)
y_kmeans = kmeans_PCs.predict(DF_embedded)

***

***

***

#### Variables in this notebook

> still under construction

| Variable             | Usage                    | Data type   |
| :------------------- | :----------------------- | :-----------|
| `clusters_df`        | Whole data               | DF          |
| `energy`             | energies col             | Series      |
| `coord`              | coordinate data          | DF          |
| `coord_train`        | x for training           | array       |
| `coord_test`         | x for testing            | array       |
| `energy_train`       | y for training           | array       |
| `energy_test`        | y for testing            | array       |
| `model`              | Linear Reg               | ***         |
| `r_sq`               | $ R^2 $-score            | float       |
| `energy_pred`        | predicted energies       | array       |
| `predictions`        | pred comparison          | DF          |
| `pca`                | PCA analysis obejct      | pca         |
| `PCs_coord`          | PCs of coordinates       | array?      |
| `PCs_df`             | PCs of coordinates       | DF          |
| `PCsE_df`            | PCs and energies         | DF          |
| `kmeans_ED`          | K-means object E+dipole  | kmeans      |
| `ED_kmeans_res`      | K-means prediction       | array       |
| `centers_ED`         | E+dipol centers for plot | array       |
| `kmeans_PCs`         | K-means object PCA       | kmeans      |
| `PCs_kmeans`         | K-means pred from PCs    | array       |
| `centers_PCs`        | PCs centers for plot     | array       |
| `kmeans_PCE`         | K-means object PCs+E     | kmeans      |
| `PCE_kmeans`         | K-means pred from PCs+E  | array       |
| `centers_PCE`        | E+PCA centers for plot   | array       |
| `DF_embedded`        | Stuff to t-SNE           | ???         |
| `centers_tSNE`       | E+PCA centers for t-SNE  | array       |


***

> We propably want to develop a functionality to actually copy wanted data into a nice folder
- eg. all clusters belonging to a cluster into a same folder
- 

In [ ]:
PCs_df.sample(5)

In [ ]:
PCE_kmeans.shape

In [ ]:
clusters_df.shape

***

***

***

***

***

***